## Búsqueda semántica 1

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents smolagents[litellm] python-dotenv

In [ ]:
!python.exe -m pip install --upgrade pip

### Cargar la clave de hugging face desde el fichero .env

In [1]:
import pprint

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### Bibliotecas específicas para postgres con pgvector

In [ ]:
!pip install psycopg2-binary numpy

### Embeddings

In [ ]:
!pip install transformers torch

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

# Cargar modelo y tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_embedding = AutoModel.from_pretrained(model_name)

e:\JAZ\AI3\ACADEMY\RETOSMOLAGENT\SMOL_AGENT_FERRETERIA\JAZ\EJEMPLO1\.VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def get_embedding(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model_embedding(**inputs)
    # Obtener la representación del [CLS] token
    embedding = outputs.last_hidden_state[:, 0, :]
    return embedding.numpy()

In [ ]:
def search_similar(query, limit):
    query_vector = get_embedding(query)
    result = db.search_similar(query_vector[0], limit=3)

    pprint.pprint('Resultados: ')
    pprint.pprint(result)

### Uso de postgres_vector.py

In [5]:
from postgres_vector import PostgresVector

db = PostgresVector()

In [6]:
db.connect()

db.create_vector_extension()
db.create_items_table(dimension=384)  # Ajustar según la dimensión de tus embeddings

True

In [7]:
name = 'Martillo'
descripcion = 'Martillo herramienta para golpear y clavar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Destornillador'
descripcion = 'Destornillador herramienta que sirve para atornillar y desatornillar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Alicates'
descripcion = 'Alicates herramienta para cualquier uso'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Llave inglesa'
descripcion = 'Llave inglesa herramienta para apretar y desapretar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Tijeras'
descripcion = 'Tijeras herramienta para cortar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Clavo'
descripcion = 'Clavo que sirve para clavar objetos'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Tornillo'
descripcion = 'Tornillo que sirve para sujetar objetos'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Tuerca'
descripcion = 'Tuerca que sirve para sujetar objetos'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])


name = '¿Cómo te llamas?'
descripcion = '¿Cómo te llamas? pregunta para saber el nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = '¿Cuál es tu nombre?'
descripcion = '¿Cuál es tu nombre? pregunta para saber el nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Dime tu nombre'
descripcion = 'Dime tu nombre pregunta para saber el nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = '¿Cómo estás?'
descripcion =  '¿Cómo estás? pregunta para saber si estás bien o no'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Me encuentro bien'
descripcion = 'Me encuentro bien respuesta para decir que estoy bien'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Estoy enfermo'
descripcion = 'Estoy enfermo respuesta para decir que estoy '
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'No me duele nada'
descripcion = 'No me duele nada respuesta para decir que estoy bien'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

True

In [8]:
name = 'qué puedo usar para clavar un clavo'
descripcion = name
query_vector = get_embedding(descripcion)
resultados = db.search_similar(query_vector[0], limit=3)

pprint.pprint('Resultados: ')
pprint.pprint(resultados)

'Resultados: '
[(21, 'Clavo', 'Clavo que sirve para clavar objetos', 0.09040693576683156),
 (23, 'Tuerca', 'Tuerca que sirve para sujetar objetos', 0.168769504095854),
 (22,
  'Tornillo',
  'Tornillo que sirve para sujetar objetos',
  0.17218497500200747)]


In [ ]:
# Actualizar un item
db.update_item(item_id=1, description="Martillo de acero inoxidable")

# Eliminar un item
db.delete_item(item_id=2)

In [ ]:
db.delete_all_items()

In [20]:
db.disconnect()

In [ ]:
pprint.pprint(embedding[0])

In [ ]:
name = '¿Cómo te llamas?'
descripcion = '¿Cómo te llamas?'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = '¿Cuál es tu nombre?'
descripcion = '¿Cuál es tu nombre?'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Dime tu nombre'
descripcion = 'Dime tu nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

In [18]:
# Ejemplo de uso con el patrón with
with PostgresVector() as db:
    # Crear la extensión y tabla si no existen
    db.create_vector_extension()
    db.create_items_table(dimension=3)  # Ajustar según la dimensión de tus embeddings
    
    # Insertar un item
    embedding = [0.1, 0.2, 0.3]  # Vector de embedding
    db.insert_item("Martillo", "Herramienta para clavar", embedding)
    
    # Buscar items similares
    query_vector = [0.15, 0.25, 0.35]
    resultados = db.search_similar(query_vector, limit=5)
    
    # Actualizar un item
    db.update_item(item_id=1, description="Martillo de acero inoxidable")
    
    # Eliminar un item
    db.delete_item(item_id=2)
    

Descartar LiteLLMModel porque falla a la hora de finalizar en el step correcto.

In [34]:
'''
from smolagents import LiteLLMModel, CodeAgent, tool

# Configurar el modelo
model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = LiteLLMModel(model_id='huggingface/'+model_id)
'''

Usar en su lugar HFApiModel que sí termina en el step adecuado.

In [78]:
from smolagents import HfApiModel, CodeAgent, tool

model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = HfApiModel(model_id=model_id)